In [2]:
print("All OK!")

All OK!


In [3]:
#importing required libraries
from langchain import PromptTemplate #for templating
from langchain.chains import RetrievalQA #for retrieving ans
from langchain.embeddings import HuggingFaceEmbeddings #for creating embeddings
from langchain.vectorstores import Pinecone #for storing vectordb
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader #for pdf and dir loading
from langchain.text_splitter import RecursiveCharacterTextSplitter #for creating chunks
from langchain.prompts import PromptTemplate #for prompting
from langchain.llms import CTransformers #quantized model

f:\Python_Classes\NLP_alldata\GenerativeAI\Day012_Day013_E2E_Project_MedicalChatBot\GenAI_E2E_Medical_Chatbot_Llama2\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [17]:
#setting up pinecone database
PINECONE_API_KEY="34ce10a9-bb10-4fbb-854c-7ae3799d8d07"
PINECONE_API_ENV="gcp-starter"

In [5]:
#loading data
#Extracting data from pdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf", #loads all pdf files from thd directory
                    loader_cls=PyPDFLoader) #lodar class is PyPDFLoader to load the pdfs
    documents = loader.load()
    return documents


In [6]:
extracted_data = load_pdf("data/")

In [7]:
#extracted_data

In [8]:
#Creating text chunks from extracted data
#creating custom function to create chunk from input data
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [9]:
#creating chunks using our custom function
text_chunks = text_split(extracted_data=extracted_data)
print("length of my chunks: ",len(text_chunks))

length of my chunks:  7020


In [10]:
#text_chunks

In [11]:
#Converting chunks to Vector Embeddings
#Creating custom function to download embedding model
#! pip install -U sentence-transformers
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [12]:
#download the embedding model
embeddings = download_hugging_face_embeddings()

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\pydot.py:17: UserWarning: `pydot` could not import `dot_parser`, so `pydot` will be unable to parse DOT files. The error was:  No module named 'pyparsing'
  warnings.warn(
f:\Python_Classes\NLP_alldata\GenerativeAI\Day012_Day013_E2E_Project_MedicalChatBot\GenAI_E2E_Medical_Chatbot_Llama2\mchatbot\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [14]:
import sys
print(sys.version)


3.9.18 (main, Sep 11 2023, 14:09:26) [MSC v.1916 64 bit (AMD64)]


In [15]:
#testing/querying from embeddings
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result)) #o/p is vector embedding of Hello world

Length 384


In [21]:
#Initializing the Pinecone
#This method is deprecated and not working anymore
'''pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)'''
#use below method to initialize Pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("medical-chatbot")

In [ ]:
#This method is deprecated and not working anymore
#docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

for text_chunk, embedding in zip(text_chunks, embeddings):
    index.upsert(ids=[text_chunk.page_content], vectors=[embedding])

In [30]:
#docsearch = pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index)

In [35]:
# Upsert vectors into the Pinecone index
for i, (text_chunk, embedding) in enumerate(zip(text_chunks, embeddings)):
    index.upsert(ids=[str(i)], vectors=[embedding])

PineconeApiTypeError: Invalid type for variable '0'. Required value type is float and passed type was Transformer at ['values'][0]

In [ ]:
#quering from the embeddings
query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

In [37]:
#creating prompt template
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [38]:
#designing prompt and chaining it
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [44]:
# setting up llm
llm=CTransformers(model="model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

OSError: [WinError -1073741795] Windows Error 0xc000001d

In [43]:
#finetuning responce with llm
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

NameError: name 'llm' is not defined

In [42]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

NameError: name 'qa' is not defined